In [31]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans

SEED = 31

In [2]:
data = pd.read_pickle("simplified_data_tim.pkl")

In [6]:
cols = data.columns
mask = cols.str.isnumeric()

yy_cols = []
for col, mm in zip(cols, mask): 
    if mm: yy_cols.append(col)

In [26]:
count_dict = {}
for col in data[yy_cols]:
    count_dict[col] = data[col].sum()

count_dict = sorted(count_dict.items(), key=lambda x: x[1], reverse=True)

1000

In [177]:
top1000cols = [int(x) for x in list(pd.read_csv("data/GLC24_PA_metadata_train.csv")["speciesId"].value_counts().to_dict().keys())][:1000]            
top500cols  = [int(x) for x in list(pd.read_csv("data/GLC24_PA_metadata_train.csv")["speciesId"].value_counts().to_dict().keys())][:500]            
next500cols = [int(x) for x in list(pd.read_csv("data/GLC24_PA_metadata_train.csv")["speciesId"].value_counts().to_dict().keys())][500:1000]            

In [179]:
yy1000 = data[top1000cols]
yy1000.head()

,540,4397,254,4499,10317,2885,1964,10600,10073,11140,...,6772,8633,6250,2237,4795,8590,1694,4924,1236,1466
surveyId,,,,,,,,,,,,,,,,,,,,,
212,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
222,True,False,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
243,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
324,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
333,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [180]:
print(yy1000[yy1000.columns[0]].sum())
print(yy1000[yy1000.columns[1]].sum())
print((yy1000[yy1000.columns[1]] & yy1000.columns[0]).sum())

21130
19124
0


In [181]:
yy1000[yy1000.columns[0]].head()
# (yy1000[yy1000.columns[1]] * yy1000.columns[0]).head(50)

surveyId
212    False
222     True
243    False
324    False
333    False
Name: 540, dtype: bool

In [182]:
(yy1000[yy1000.columns[0]] & yy1000[yy1000.columns[1]]).sum()

10226

In [183]:
def bayes_theorem(df, index_0, index_1):
    cols = df.columns
    joint_occurence = (df[cols[index_0]] & df[cols[index_1]]).sum()
    p_ba = joint_occurence / df[cols[index_1]].sum()
    p_b = df[cols[index_0]].sum() / df.shape[0]
    p_a = df[cols[index_1]].sum() / df.shape[0]
    return p_ba, (p_ba * p_a / p_b)

def compare_occurence(df, index_0, index_1, threshold_min:float = 0.9, threshold_max:float = 1.0):
    ratio, bayes = bayes_theorem(df, index_0, index_1)
    # if ratio > threshold_min and ratio < threshold_max:
        # print(f":: {index_1} occurs in [{index_0}] {100 * ratio:.2f}% of the time")
    return True, ratio, bayes
    # return False, None, None

In [184]:
symbiosis = []
for ii in range(500):
    for jj in range(500, len(yy1000.columns)):
        truth, ratio, bayes = compare_occurence(
            yy1000, ii, jj, threshold_min=0.8, threshold_max=1.0)
        if truth: symbiosis.append([yy1000.columns[ii], yy1000.columns[jj], ratio, bayes])

symbiosis = pd.DataFrame(symbiosis, columns=["parent", "child", "occurrence", "bayes"])


In [185]:
symbiosis.head(10)

,parent,child,occurrence,bayes
0,540,2361,0.020779,0.000379
1,540,9905,0.476839,0.008282
2,540,6557,0.086957,0.001704
3,540,351,0.010526,0.000189
4,540,4639,0.074879,0.001467
5,540,3669,0.026634,0.000521
6,540,4892,0.004926,0.000095
7,540,8177,0.165854,0.003218
8,540,2110,0.328076,0.004922
9,540,3835,0.165842,0.003171


In [186]:
symbiosis.to_csv("bayes_symbiosis.csv", index=False)

In [187]:
bs = pd.read_csv("bayes_symbiosis.csv")
bs.shape

(250000, 4)

In [ ]:
yy500 = data[top500cols]
yy500[[next500cols]] = 1.0

In [232]:
#  TESTING
child = 1466
species1466 = bs[bs.child == child]
for parent, bayes in zip(species1466.parent, species1466.bayes):
    for idx in range(yy500.shape[0]):
        if yy500[parent].iloc[idx]:
            yy500.at[idx, child] = yy500[child].iloc[idx] * (1 - bayes)
    
    

KeyboardInterrupt: 

In [227]:
# yy500.head(1)[540] is False
print(yy500[540].iloc[1])
yy500.head()

True


,540,4397,254,4499,10317,2885,1964,10600,10073,11140,...,6772,8633,6250,2237,4795,8590,1694,4924,1236,1466
surveyId,,,,,,,,,,,,,,,,,,,,,
212,False,False,False,False,False,False,False,False,False,False,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
222,True,False,True,True,False,False,False,False,False,False,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
243,False,False,True,False,False,False,False,False,False,False,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
324,False,False,False,False,False,False,False,False,False,False,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
333,False,False,False,False,False,False,False,False,False,False,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
dict500 = dict(zip(next500cols, np.ones(500)))
dict500

In [207]:
yy500.head()

,540,4397,254,4499,10317,2885,1964,10600,10073,11140,...,6772,8633,6250,2237,4795,8590,1694,4924,1236,1466
surveyId,,,,,,,,,,,,,,,,,,,,,
212,False,False,False,False,False,False,False,False,False,False,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
222,True,False,True,True,False,False,False,False,False,False,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
243,False,False,True,False,False,False,False,False,False,False,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
324,False,False,False,False,False,False,False,False,False,False,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
333,False,False,False,False,False,False,False,False,False,False,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [233]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

model = RandomForestRegressor()

yy500 = data[top500cols]
yy500_1000 = data[next500cols]

xx_t, xx_v, yy_t, yy_v = train_test_split(yy500, yy500_1000, test_size=.2)